## AIHub Json Parsing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [ ]:
%pip install ray

In [1]:
import re
import os
import kss
import ray
import json
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [2]:
pwd

'D:\\AIHUB'

### Function

In [3]:
def sorted_list(path_list):
    path_list = sorted(path_list, reverse=False)
    path_list = sorted(path_list, key=len)
    
    return path_list

In [4]:
def json_file_name_list(path_list):
    
    file_name  = [glob(i, recursive = True) for i in path_list][0]
    file_name = sorted_list(file_name)
    
    return file_name

In [5]:
def train_valid_json_file_name_list(path_list):

  train_file_name, valid_file_name = [glob(i, recursive = True) if 'rain' in i
                                      else glob(i, recursive = True)
                                      for i in path_list]

  train_file_name = sorted_list(train_file_name)
  valid_file_name = sorted_list(valid_file_name)
    
  return train_file_name, valid_file_name

In [6]:
def divide_source_file_list(l, n): 
    
  for i in range(0, len(l), n): 
    yield l[i:i + n] 

In [7]:
def txt_file_name_list(source_file_nested_list, folder_corpus_type_name):

  text_file_name_list = [folder_corpus_type_name + str(i) + ".txt"
                              for i in range(len(source_file_nested_list))]
    
  return text_file_name_list

In [8]:
def post_txt_file_name_list(corpus_list):
   
  post_corpus_list = [corpus_file.replace("pro", "post")
                      for corpus_file in corpus_list]

  return post_corpus_list

In [9]:
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = re.sub(source[:2], "", source)
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외

                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)

            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

  
    return preprocessing_sentence_list

### AIHUB 요약문 및 레포트 생성 데이터

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=582)

In [10]:
path_list = ['AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/01.news_r/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/02.briefing/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/03.his_cul/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/04.paper/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/05.minute/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/06.edit/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/07.public/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/08.speech/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/10.narration/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/01.news_r/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/02.briefing/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/03.his_cul/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/04.paper/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/05.minute/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/06.edit/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/07.public/' + '**/*.json',             
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/08.speech/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/09.literature/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/10.narration/' + '**/*.json']

train_file_name_01 = glob(path_list[0], recursive = True)
train_file_name_02 = glob(path_list[1], recursive = True)
train_file_name_03 = glob(path_list[2], recursive = True)
train_file_name_04 = glob(path_list[3], recursive = True)
train_file_name_05 = glob(path_list[4], recursive = True)
train_file_name_06 = glob(path_list[5], recursive = True)
train_file_name_07 = glob(path_list[6], recursive = True)
train_file_name_08 = glob(path_list[7], recursive = True)
train_file_name_09 = glob(path_list[8], recursive = True)
train_file_name_10 = glob(path_list[9], recursive = True)
valid_file_name_01 = glob(path_list[10], recursive = True)
valid_file_name_02 = glob(path_list[11], recursive = True)
valid_file_name_03 = glob(path_list[12], recursive = True)
valid_file_name_04 = glob(path_list[13], recursive = True)
valid_file_name_05 = glob(path_list[14], recursive = True)
valid_file_name_06 = glob(path_list[15], recursive = True)
valid_file_name_07 = glob(path_list[16], recursive = True)
valid_file_name_08 = glob(path_list[17], recursive = True)
valid_file_name_09 = glob(path_list[18], recursive = True)
valid_file_name_10 = glob(path_list[19], recursive = True)  

In [11]:
the_number_of_file = len(train_file_name_01) 
print("The number of file: ", the_number_of_file)
n = the_number_of_file // 1000
print("The number of list element:", n)
train_file_nested_list_01 = list(divide_source_file_list(train_file_name_01, n))
train_file_nested_list_02 = list(divide_source_file_list(train_file_name_02, n))
train_file_nested_list_03 = list(divide_source_file_list(train_file_name_03, n))
train_file_nested_list_04 = list(divide_source_file_list(train_file_name_04, n))
train_file_nested_list_05 = list(divide_source_file_list(train_file_name_05, n))
train_file_nested_list_06 = list(divide_source_file_list(train_file_name_06, n))
train_file_nested_list_07 = list(divide_source_file_list(train_file_name_07, n))
train_file_nested_list_08 = list(divide_source_file_list(train_file_name_08, n))
train_file_nested_list_09 = list(divide_source_file_list(train_file_name_09, n))
train_file_nested_list_10 = list(divide_source_file_list(train_file_name_10, n))
valid_file_nested_list_01 = list(divide_source_file_list(valid_file_name_01, n))
valid_file_nested_list_02 = list(divide_source_file_list(valid_file_name_02, n))
valid_file_nested_list_03 = list(divide_source_file_list(valid_file_name_03, n))
valid_file_nested_list_04 = list(divide_source_file_list(valid_file_name_04, n))
valid_file_nested_list_05 = list(divide_source_file_list(valid_file_name_05, n))
valid_file_nested_list_06 = list(divide_source_file_list(valid_file_name_06, n))
valid_file_nested_list_07 = list(divide_source_file_list(valid_file_name_07, n))
valid_file_nested_list_08 = list(divide_source_file_list(valid_file_name_08, n))
valid_file_nested_list_09 = list(divide_source_file_list(valid_file_name_09, n))
valid_file_nested_list_10 = list(divide_source_file_list(valid_file_name_10, n))

The number of file:  21600
The number of list element: 21


In [13]:
train_text_file_name_list_01 = txt_file_name_list(train_file_nested_list_01,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_01.news_r_train_")
train_text_file_name_list_02 = txt_file_name_list(train_file_nested_list_02,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_02.briefing_train_")
train_text_file_name_list_03 = txt_file_name_list(train_file_nested_list_03,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_03.his_cul_train_")
train_text_file_name_list_04 = txt_file_name_list(train_file_nested_list_04,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_04.paper_train_")
train_text_file_name_list_05 = txt_file_name_list(train_file_nested_list_05,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_05.minute_train_")
train_text_file_name_list_06 = txt_file_name_list(train_file_nested_list_06,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_06.edit_train_")
train_text_file_name_list_07 = txt_file_name_list(train_file_nested_list_07,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_07.public_train_")
train_text_file_name_list_08 = txt_file_name_list(train_file_nested_list_08,
                                                               "explorationo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_08.speech_train_")
train_text_file_name_list_09 = txt_file_name_list(train_file_nested_list_09,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_09.literature_train_")
train_text_file_name_list_10 = txt_file_name_list(train_file_nested_list_10,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_10.narration_train_")
valid_text_file_name_list_01 = txt_file_name_list(valid_file_nested_list_01,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_01.news_r_valid_")
valid_text_file_name_list_02 = txt_file_name_list(valid_file_nested_list_02,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_02.briefing_valid_")
valid_text_file_name_list_03 = txt_file_name_list(valid_file_nested_list_03,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_03.his_cul_valid_")
valid_text_file_name_list_04 = txt_file_name_list(valid_file_nested_list_04,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_04.paper_valid_")
valid_text_file_name_list_05 = txt_file_name_list(valid_file_nested_list_05,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_05.minute_valid_")
valid_text_file_name_list_06 = txt_file_name_list(valid_file_nested_list_06,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_06.edit_valid_")
valid_text_file_name_list_07 = txt_file_name_list(valid_file_nested_list_07,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_07.public_valid_")
valid_text_file_name_list_08 = txt_file_name_list(valid_file_nested_list_08,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_08.speech_valid_")
valid_text_file_name_list_09 = txt_file_name_list(valid_file_nested_list_09,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_09.literature_valid_")
valid_text_file_name_list_10 = txt_file_name_list(valid_file_nested_list_10,
                                                               "exploration/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_10.narration_valid_")

the_numer_of_txt_file = len(train_text_file_name_list_01) + len(train_text_file_name_list_02) + \
    len(train_text_file_name_list_03) + len(train_text_file_name_list_04) + \
    len(train_text_file_name_list_05) + len(train_text_file_name_list_06) + \
    len(train_text_file_name_list_07) + len(train_text_file_name_list_08) + \
    len(train_text_file_name_list_09) + len(train_text_file_name_list_10) +  \
    len(valid_text_file_name_list_01) + len(valid_text_file_name_list_02) + \
    len(valid_text_file_name_list_03) + len(valid_text_file_name_list_04) + \
    len(valid_text_file_name_list_05) + len(valid_text_file_name_list_06) + \
    len(valid_text_file_name_list_07) + len(valid_text_file_name_list_08) + \
    len(valid_text_file_name_list_09) + len(valid_text_file_name_list_10)
print("The number of txt file:", the_numer_of_txt_file )

The number of txt file: 7868


In [17]:
def list_length_checker(source_file_nested_list, batch_size):
    
    the_number_of_total_txt_file = 0
    the_number_of_txt_file_list = []
    temp_nested_index = []
    source_list = []
    
    for i in range(len(source_file_nested_list)):   
        source_file_list = source_file_nested_list[i]
        temp_index = []
        for j in range(len(source_file_list)):
            
            if type(source_file_list) == str:
                source_file = source_file_list

            elif type(source_file_list) == list:
                source_file = source_file_list[j]
            
            with open(source_file, 'r', encoding='utf-8') as one_json_file:
                one_json_sample = json.load(one_json_file) 

            source = list(one_json_sample['Meta(Refine)']['passage'])
            
            if (len(source_list) >= batch_size) or \
            (i == (len(source_file_nested_list) -1) and j == (len(source_file_list) -1)):
              num += 1
              print(str(num), end=" ")    

              source_list = []
              source_list.append(source)
            
            elif 

            else:
              source_list.append(source)  

                
            the_number_of_txt_file = 1

            if len(source_list) >= batch_size:
                print("File:", source_file_nested_list[i][0] + "  ~  " + source_file)    
                print("Index:", i, "  ", "Length of Source List:", len(source_list), \
                    "  ", "The number of txt file:", the_number_of_txt_file, "\n")
                the_number_of_txt_file_list.append(the_number_of_txt_file)
                the_number_of_total_txt_file  += the_number_of_txt_file
            else:
                the_number_of_total_txt_file  += 1
                the_number_of_txt_file_list.append(1)
                if j > 0 and the_number_of_file % j == 0:
                    temp_index.append(j)
                    try:
                        if j == temp_nested_index[0][0] and len(temp_nested_index) <= 1:
                            print("[For Example]")
                            print("This is not subject of batch. It's small source list.")                            
                            print("File:", source_file)
                            print("Length of Source List:", len(source_list), 
                                    "  ", "The number of txt file:", 1, "\n") 
                    except:
                        pass

    print("Batch Size:", batch_size)
    print("The number of txt file:", the_number_of_total_txt_file)
    
    return the_number_of_total_txt_file, the_number_of_txt_file_list

In [ ]:
def make_corpus_txt_with_batch_list(source_file_nested_list,
                                    text_file_name_list,
                                    batch_size, the_number_of_total_txt_file_list):

  print("[Size]")
  print("The numnber of preprocessing corpus: " + str(sum(the_number_of_total_txt_file_list)))
  print("\n[Order]")
  num = 0
  source_list = []

  for i in range(len(source_file_nested_list)):
    source_file_list = source_file_nested_list[i]
    
    for j in range(len(source_file_list)):
      
      if type(source_file_list) == str:
        source_file = source_file_list

      elif type(source_file_list) == list:
        source_file = source_file_list[j]

      with open(source_file, 'r', encoding='utf-8') as one_json_file:
        one_json_sample = json.load(one_json_file)

      source = [one_json_sample['Meta(Refine)']['passage']]
      
      if len(source_list) >= batch_size or \
     (i == (len(source_file_nested_list) -1) and j == (len(source_file_list) -1)):
          num += 1
          print(str(num), end=" ")  
          
          with open(os.path.join('AIHUB_corpus/' + text_file_name_list[i][:-4] + "_" + str(num) + ".txt"), "a", encoding='utf-8') as fp:        
              fp.write("\n".join(source_list))   
    
          source_list = []
          source_list.append(source)
            
       else:
          source_list.append(source)          

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_01, the_number_of_train_txt_file_list_01 = list_length_checker(train_file_nested_list_01, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_02, the_number_of_train_txt_file_list_02 = list_length_checker(train_file_nested_list_02, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_03, the_number_of_train_txt_file_list_03 = list_length_checker(train_file_nested_list_03, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_04, the_number_of_train_txt_file_list_04 = list_length_checker(train_file_nested_list_04, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_05, the_number_of_train_txt_file_list_05 = list_length_checker(train_file_nested_list_05, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_06, the_number_of_train_txt_file_list_06 = list_length_checker(train_file_nested_list_06, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_07, the_number_of_train_txt_file_list_07 = list_length_checker(train_file_nested_list_07, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_08, the_number_of_train_txt_file_list_08 = list_length_checker(train_file_nested_list_08, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_09, the_number_of_train_txt_file_list_09 = list_length_checker(train_file_nested_list_09, batch_size)

In [ ]:
batch_size = 1000
the_number_of_train_txt_file_10, the_number_of_train_txt_file_list_10 = list_length_checker(train_file_nested_list_10, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_01, the_number_of_valid_txt_file_list_01 = list_length_checker(valid_file_nested_list_01, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_02, the_number_of_valid_txt_file_list_02 = list_length_checker(valid_file_nested_list_02, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_03, the_number_of_valid_txt_file_list_03 = list_length_checker(valid_file_nested_list_03, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_04, the_number_of_valid_txt_file_list_04 = list_length_checker(valid_file_nested_list_04, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_05, the_number_of_valid_txt_file_list_05 = list_length_checker(valid_file_nested_list_05, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_06, the_number_of_valid_txt_file_list_06 = list_length_checker(valid_file_nested_list_06, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_07, the_number_of_valid_txt_file_list_07 = list_length_checker(valid_file_nested_list_07, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_08, the_number_of_valid_txt_file_list_08 = list_length_checker(valid_file_nested_list_08, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_09, the_number_of_valid_txt_file_list_09 = list_length_checker(valid_file_nested_list_09, batch_size)

In [ ]:
batch_size = 1000
the_number_of_valid_txt_file_10, the_number_of_valid_txt_file_list_10 = list_length_checker(valid_file_nested_list_10, batch_size)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(train_file_nested_list_01, train_text_file_name_list_01, batch_size, the_number_of_train_txt_file_list_01)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(train_file_nested_list_02, train_text_file_name_list_02, batch_size, the_number_of_train_txt_file_list_02)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(train_file_nested_list_03, train_text_file_name_list_03, batch_size, the_number_of_train_txt_file_list_03)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(train_file_nested_list_04, train_text_file_name_list_04, batch_size, the_number_of_train_txt_file_list_04)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(train_file_nested_list_05, train_text_file_name_list_05, batch_size, the_number_of_train_txt_file_list_05)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(train_file_nested_list_06, train_text_file_name_list_06, batch_size, the_number_of_train_txt_file_list_06)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(train_file_nested_list_07, train_text_file_name_list_07, batch_size, the_number_of_train_txt_file_list_07)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(train_file_nested_list_08, train_text_file_name_list_08, batch_size, the_number_of_train_txt_file_list_08)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(train_file_nested_list_09, train_text_file_name_list_09, batch_size, the_number_of_train_txt_file_list_09)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(train_file_nested_list_10, train_text_file_name_list_10, batch_size, the_number_of_train_txt_file_list_10)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(valid_file_nested_list_01, valid_text_file_name_list_01, batch_size, the_number_of_valid_txt_file_list_01)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(valid_file_nested_list_02, valid_text_file_name_list_02, batch_size, the_number_of_valid_txt_file_list_02)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(valid_file_nested_list_03, valid_text_file_name_list_03, batch_size, the_number_of_valid_txt_file_list_03)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(valid_file_nested_list_04, valid_text_file_name_list_04, batch_size, the_number_of_valid_txt_file_list_04)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(valid_file_nested_list_05, valid_text_file_name_list_05, batch_size, the_number_of_valid_txt_file_list_05)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(valid_file_nested_list_06, valid_text_file_name_list_06, batch_size, the_number_of_valid_txt_file_list_06)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(valid_file_nested_list_07, valid_text_file_name_list_07, batch_size, the_number_of_valid_txt_file_list_07)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(valid_file_nested_list_08, valid_text_file_name_list_08, batch_size, the_number_of_valid_txt_file_list_08)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(valid_file_nested_list_09, valid_text_file_name_list_09, batch_size, the_number_of_valid_txt_file_list_09)

In [ ]:
batch_size = 1000
make_corpus_txt_with_batch_list(valid_file_nested_list_10, valid_text_file_name_list_10, batch_size, the_number_of_valid_txt_file_list_10)

In [ ]:
pro_total_corpus_list = glob("AIHUB_corpus/exploration/summary_and_report_generation_data_pro/AIHUB_summary_and_report_generation_data_" +"*.txt")

In [ ]:
len(pro_total_corpus_list)

In [ ]:
post_total_corpus_list = post_txt_file_name_list(pro_total_corpus_list)

In [ ]:
line_list = []
line_num = 0
with open(pro_total_corpus_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines() 
    for line in lines:
        line_num += 1
        if line_num <= 1:
           line_list.append(line)
for line in line_list:
    print(line, end="\n\n")

In [ ]:
line_list = []
line_num = 0
with open(pro_total_corpus_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
    for line in lines:
        line_num += 1
        if line_num <= 1:  
            sentences = formal_preprocessing_text(line)
            for sentence in sentences:
                line_list.append(sentence) 
            
for line in line_list:
    print(line, end="\n\n")

In [ ]:
ray.init(num_cpus = 4)

@ray.remote

def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = re.sub(source[:2], "", source)
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외

                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)

            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

  
    return preprocessing_sentence_list

In [ ]:
topic_name_list = ['01.news_r_valid', '02.briefing', '03.his_cul', '04.paper', '05.minute', '06.edit', '07.public',
                  '08.speech', '09.literature', '10.narration']

In [ ]:
pro_total_corpus_list_train_01 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[0] + "_train" + "*.txt")
post_total_corpus_list_train_01 = post_txt_file_name_list(pro_total_corpus_list_train_01)

pro_total_corpus_list_train_02 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[1] + "_train" + "*.txt")
post_total_corpus_list_train_02 = post_txt_file_name_list(pro_total_corpus_list_train_02)

pro_total_corpus_list_train_03 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[2] + "_train" + "*.txt")
post_total_corpus_list_train_03 = post_txt_file_name_list(pro_total_corpus_list_train_03)

pro_total_corpus_list_train_04 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[3] + "_train" + "*.txt")
post_total_corpus_list_train_04 = post_txt_file_name_list(pro_total_corpus_list_train_04)

pro_total_corpus_list_train_05 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[4] + "_train" + "*.txt")
post_total_corpus_list_train_05 = post_txt_file_name_list(pro_total_corpus_list_train_05)

pro_total_corpus_list_train_06 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[5] + "_train" + "*.txt")
post_total_corpus_list_train_06 = post_txt_file_name_list(pro_total_corpus_list_train_06)

pro_total_corpus_list_train_07 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[6] + "_train" + "*.txt")
post_total_corpus_list_train_07 = post_txt_file_name_list(pro_total_corpus_list_train_07)

pro_total_corpus_list_train_08 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[7] + "_train" + "*.txt")
post_total_corpus_list_train_08 = post_txt_file_name_list(pro_total_corpus_list_train_08)

pro_total_corpus_list_train_09 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[8] + "_train" + "*.txt")
post_total_corpus_list_train_09 = post_txt_file_name_list(pro_total_corpus_list_train_09)

pro_total_corpus_list_train_10 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[9] + "_train" + "*.txt")
post_total_corpus_list_train_10 = post_txt_file_name_list(pro_total_corpus_list_train_10)


pro_total_corpus_list_valid_01 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[0] + "_valid" + "*.txt")
post_total_corpus_list_valid_01 = post_txt_file_name_list(pro_total_corpus_list_valid_01)

pro_total_corpus_list_valid_02 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[1] + "_valid" + "*.txt")
post_total_corpus_list_valid_02 = post_txt_file_name_list(pro_total_corpus_list_valid_02)

pro_total_corpus_list_valid_03 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[2] + "_valid" + "*.txt")
post_total_corpus_list_valid_03 = post_txt_file_name_list(pro_total_corpus_list_valid_03)

pro_total_corpus_list_valid_04 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[3] + "_valid" + "*.txt")
post_total_corpus_list_valid_04 = post_txt_file_name_list(pro_total_corpus_list_valid_04)

pro_total_corpus_list_valid_05 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[4] + "_valid" + "*.txt")
post_total_corpus_list_valid_05 = post_txt_file_name_list(pro_total_corpus_list_valid_05)

pro_total_corpus_list_valid_06 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[5] + "_valid" + "*.txt")
post_total_corpus_list_valid_06 = post_txt_file_name_list(pro_total_corpus_list_valid_06)

pro_total_corpus_list_valid_07 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[6] + "_valid" + "*.txt")
post_total_corpus_list_valid_07 = post_txt_file_name_list(pro_total_corpus_list_valid_07)

pro_total_corpus_list_valid_08 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[7] + "_valid" + "*.txt")
post_total_corpus_list_valid_08 = post_txt_file_name_list(pro_total_corpus_list_valid_08)

pro_total_corpus_list_valid_09 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[8] + "_valid" + "*.txt")
post_total_corpus_list_valid_09 = post_txt_file_name_list(pro_total_corpus_list_valid_09)

pro_total_corpus_list_valid_10 = glob("AIHUB_corpus/exploration/web_data_based_korean_corpus_data_pro/AIHUB_web_data_based_korean_corpus_data_" + \
                                topic_name_list[9] + "_valid" + "*.txt")
post_total_corpus_list_valid_10 = post_txt_file_name_list(pro_total_corpus_list_valid_10)

In [ ]:
def preprocessing_corpus_txt(pro_total_corpus_list, post_total_corpus_list)
    print("[Size]")
    print("The number of preprocessing corpus: " + str(len(pro_total_corpus_list)))
    print("\n[Order]")
    num = 0
    process_num = 10    

    for pro, post in zip(pro_total_corpus_list, post_total_corpus_list):

        sentence_list = []

        with open(pro, 'r', encoding='utf-8') as f:
            lines = f.read().splitlines() 
            nested_lines_num = len(lines) // process_num
            for i in range(nested_lines_num - 1):
                start_line = process_num * i
                end_line = process_num * (i+1)
                futures = [formal_preprocessing_text.remote(lines[start_line:end_line][j]) for j in range(process_num)]
                results = ray.get(futures)

                if i == nested_lines_num - 2:
                    futures = [formal_preprocessing_text.remote(lines[end_line:][j]) for j in range(len(lines) - end_line)]
                    results = ray.get(futures)

                sentences = list(chain.from_iterable(results))
                sentence_list.append(sentences)

        sentence_list = list(chain.from_iterable(sentence_list))

        num += 1
        print(str(num), end=" ")  

        with open(post, 'a', encoding='utf-8') as fp:
            fp.write("\n".join(sentence_list))

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_train_01, post_total_corpus_list_train_01)

In [ ]:
preprocessing_corpus_txte(pro_total_corpus_list_train_02, post_total_corpus_list_train_02)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_train_03, post_total_corpus_list_train_03)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_train_04, post_total_corpus_list_train_04)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_train_05, post_total_corpus_list_train_05)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_train_06, post_total_corpus_list_train_06)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_train_07, post_total_corpus_list_train_07)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_train_08, post_total_corpus_list_train_08)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_train_09, post_total_corpus_list_train_09)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_train_10, post_total_corpus_list_train_10)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_valid_01, post_total_corpus_list_valid_01)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_valid_02, post_total_corpus_list_valid_02)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_valid_03, post_total_corpus_list_valid_03)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_valid_04, post_total_corpus_list_valid_04)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_valid_05, post_total_corpus_list_valid_05)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_valid_06, post_total_corpus_list_valid_06)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_valid_07, post_total_corpus_list_valid_07)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_valid_08, post_total_corpus_list_valid_08)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_valid_09, post_total_corpus_list_valid_09)

In [ ]:
preprocessing_corpus_txt(pro_total_corpus_list_valid_10, post_total_corpus_list_valid_10)

In [ ]:
ray.shutdown()

In [ ]:
corpus_list = glob("AIHUB_corpus/exploration/summary_and_report_generation_data_post/AIHUB_summary_and_report_generation_data_" +"*.txt")

In [ ]:
for i in range(len(topic_name_list)):
    with open('AIHUB_corpus/duplicate/AIHUB_summary_and_report_generation_data_'+ topic_name_list[i] + '.txt', 'w') as f:
        topic_corpus_list = [j if topic_name_list[i] in j else None for j in corpust_list]
        for corpus in topic_corpus_list:
            with open(corpus) as text:
                for line in text:
                    f.write(line)

In [ ]:
with open('AIHUB_corpus/AIHUB_summary_and_report_generation_data.txt', 'w', encoding='utf-8') as f1:
    with open('AIHUB_corpus/duplicate/AIHUB_summary_and_report_generation_data.txt', encoding='utf-8') as f2:
        lines = f2.read().splitlines()
        single_sentence_dict = dict.fromkeys(lines)
        single_sentence_list = list(single_sentence_dict)
        f1.write("\n".join(single_sentence_list))